## Sommaire: <a class="anchor" id="sommaire"></a>
* [Sommaire](#sommaire)
* [Package Loading](#package)
* [Model selection](#model_selection)
    * [FD001](#fd001)
    * [FD002](#fd002)
    * [FD003](#fd003)
    * [FD004](#fd004)

## Package's loading <a id='package'></a>

In [1]:
from __future__ import print_function
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
import warnings
import tensorflow as tf
import seaborn as sns
import sklearn
import random
import math
import time



from lime.lime_tabular import RecurrentTabularExplainer
from tqdm import tqdm
from tqdm import tqdm
from sklearn.metrics import mean_squared_error, r2_score 
from sklearn.model_selection import GroupKFold
from sklearn import preprocessing
from keras import backend as K
from sklearn.preprocessing import MinMaxScaler , StandardScaler
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Activation, GRU
from scipy import optimize
from tensorflow.keras import optimizers


from sp_modif.model_function import *
from sp_modif.methods import *
from sp_modif.data_prep import *
from sp_modif.evaluator import *
from sp_modif.SHAP import *
from sp_modif.L2X import *
from methods import *

%matplotlib inline
warnings.filterwarnings('ignore')

SEED = 0
def set_seed(seed=SEED):
    os.environ['PYTHONHASHSEED'] = str(SEED)
    random.seed(SEED)
    np.random.seed(SEED)
    tf.random.set_seed(SEED)

# Appeler la fonction pour fixer le seed
set_seed(SEED)

### FD004 <a class="anchor" id="fd004"></a>

In [2]:
# Load data
# Load data and preprocess
with tf.device('/device:GPU:0'):
    train, test, y_test = prepare_data('FD004.txt')
    print(train.shape, test.shape, y_test.shape)
    sensor_names = ['T20','T24','T30','T50','P20','P15','P30','Nf','Nc','epr','Ps30','phi',
                    'NRf','NRc','BPR','farB','htBleed','Nf_dmd','PCNfR_dmd','W31','W32']

    # remaining_sensors = ["Ps30", "Nf", "phi", "BPR", "farB"] # selection based on main_notebook
    remaining_sensors = ["Ps30", "Nf", "phi"]

    drop_sensors = [element for element in sensor_names if element not in remaining_sensors]

rul_piecewise = 120
train['RUL'].clip(upper=rul_piecewise, inplace=True)

(61249, 27) (41214, 26) (248, 1)


In [3]:
# 1layers
from tensorflow.keras.callbacks import EarlyStopping
def model_lstm_1layer(input_shape, nodes_per_layer, dropout, activation):
    
    cb = keras.callbacks.EarlyStopping(monitor='loss', patience=4)
    model = Sequential()
    model.add(LSTM(units = nodes_per_layer, activation=activation, 
                  input_shape=input_shape))
    model.add(Dropout(dropout))
    model.add(Dense(256))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error',
                  optimizer=Adam(learning_rate=0.001))
    # model.save_weights(weights_file)

    return model

In [4]:
%%time

# RMSE	std_RMSE	S_score	std_S_score	MSE	std_MSE	nodes	dropout	activation	batch_size	TW
# 1	15.449126	0.0	1.242658e+03	0.0	238.675491	0.0	[64]	0.2	tanh	32	40
# RMSE	std_RMSE	S_score	std_S_score	MSE	std_MSE	alpha	nodes	dropout	activation	batch_size	TW
# 2	16.100150	0.0	1.752538e+03	0.0	259.214813	0.0	0.30	[64]	0.2	tanh	32	40
results = pd.DataFrame()

for SEED in range(5):  
    tf.random.set_seed(SEED)
    mse = []
    R2_val = []
    RMSE = []
    score_val = []
    
    # 0.20	[64]	0.3	tanh	32	25
    
    # parameter's sample
    weights_file = "weights_file_lstm_optimalmodel_clv.h5"
    alpha = 0.3
    sequence_length = 40
    epochs = 20
    nodes_per_layer = [64]
    dropout = 0.2
    activation = 'tanh'
    batch_size = 32
    remaining_sensors = remaining_sensors
    # create model
    input_shape = (sequence_length, len(remaining_sensors))
    model = model_lstm_1layer(input_shape, nodes_per_layer[0], dropout, activation)
    
    # Data prepration
    X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)

    # create sequences train, test
    train_array = gen_data_wrapper(X_train_interim, sequence_length,remaining_sensors)
    label_array = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'])

    test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
               for unit_nr in X_test_interim['Unit'].unique())
    
    test_array = np.concatenate(list(test_gen)).astype(np.float32)
    test_rul = rul_piecewise_fct(y_test,rul_piecewise)
    print(train_array.shape, label_array.shape, test_array.shape)
    cb = EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)
            
    # Model fitting
    with tf.device('/device:GPU:0'):
        start_time = time.time()
        weights_file = model.get_weights()
        model.set_weights(weights_file)  # reset optimizer and node weights before every training iteration
        history = model.fit(train_array, label_array,
                                validation_data=(test_array, test_rul),
                                epochs=epochs,
                                batch_size=batch_size,
                                callbacks=[cb],
                                verbose=1)
        end_time = time.time()
        
        training_time = end_time - start_time
        mse.append(history.history['val_loss'][-1])

        y_hat_val_split = model.predict(test_array)
        R2_val.append(r2_score(test_rul, y_hat_val_split))
        RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
        score_val.append(compute_s_score(test_rul, y_hat_val_split))
            
        
    #  append results
    d = {'RMSE' :np.mean(RMSE), 'std_RMSE' :np.std(RMSE),
         'S_score' :np.mean(score_val), 'std_S_score' :np.std(score_val),
         'MSE':np.mean(mse), 'std_MSE':np.std(mse),
         'nodes':str(nodes_per_layer), 'dropout':dropout, 
         'activation':activation, 'batch_size':batch_size, 'TW' : sequence_length, 
         'time': training_time}

#     results = results.append(pd.DataFrame(d, index=[0]), ignore_index=True)
    results = pd.concat([results, pd.DataFrame(d, index=[0])], ignore_index=True)
    results.to_csv('results/base/fd004.csv')


(51538, 40, 3) (51538, 1) (248, 40, 3)
Epoch 1/20

1611/1611 [==============================] - 10s 6ms/step - loss: 991.5342 - val_loss: 1473.9977
Epoch 2/20
1611/1611 [==============================] - 9s 6ms/step - loss: 448.5498 - val_loss: 1224.9937
Epoch 3/20
1611/1611 [==============================] - 9s 6ms/step - loss: 381.3220 - val_loss: 1110.1326
Epoch 4/20
1611/1611 [==============================] - 10s 6ms/step - loss: 336.9692 - val_loss: 1372.1982
Epoch 5/20
1611/1611 [==============================] - 11s 7ms/step - loss: 321.1429 - val_loss: 1383.6591
Epoch 6/20
1611/1611 [==============================] - 10s 6ms/step - loss: 302.6805 - val_loss: 1355.4508
Epoch 7/20
8/8 [==============================] - 0s 2ms/step
(51538, 40, 3) (51538, 1) (248, 40, 3)
Epoch 1/20
1611/1611 [==============================] - 12s 7ms/step - loss: 1025.2651 - val_loss: 1351.9329
Epoch 2/20
1611/1611 [==============================] - 11s 7ms/step - loss: 448.1556 - val_loss: 1190.

In [5]:
results

,RMSE,std_RMSE,S_score,std_S_score,MSE,std_MSE,nodes,dropout,activation,batch_size,TW,time
0,33.318652,0.0,119058.063626,0.0,1252.416138,0.0,[64],0.2,tanh,32,40,69.976216
1,18.751287,0.0,3837.726324,0.0,521.260254,0.0,[64],0.2,tanh,32,40,122.973090
2,17.670494,0.0,3050.407626,0.0,339.261047,0.0,[64],0.2,tanh,32,40,178.253519
3,34.247137,0.0,121738.200432,0.0,1401.619263,0.0,[64],0.2,tanh,32,40,55.933658
4,34.860023,0.0,172146.644374,0.0,1639.256348,0.0,[64],0.2,tanh,32,40,95.092187


### FD002 <a class="anchor" id="fd002"></a>

In [6]:
# Load data
# Load data and preprocess
train, test, y_test = prepare_data('FD002.txt')
print(train.shape, test.shape, y_test.shape)
sensor_names = ['T20','T24','T30','T50','P20','P15','P30','Nf','Nc','epr','Ps30','phi',
                    'NRf','NRc','BPR','farB','htBleed','Nf_dmd','PCNfR_dmd','W31','W32']
# remaining_sensors = ["Ps30", "Nf", "phi", "epr"] # selection based on main_notebook [1] "Nf" [1] "Ps30" [1] "phi"
remaining_sensors = ["Ps30", "Nf", "phi"] # selection based on main_notebook [1] "Nf" [1] "Ps30" [1] "phi"
drop_sensors = [element for element in sensor_names if element not in remaining_sensors]

rul_piecewise = 125
train['RUL'].clip(upper=rul_piecewise, inplace=True)

(53759, 27) (33991, 26) (259, 1)


In [7]:
%%time

# RMSE	std_RMSE	S_score	std_S_score	MSE	std_MSE	alpha	nodes	dropout	activation	batch_size	TW
# 16.603718	0.0	1.343367e+03	0.0	275.683502	0.0	0.20	[32]	0.1	tanh	128	40
results002 = pd.DataFrame()

for SEED in range(5):  
    tf.random.set_seed(SEED)
    mse = []
    R2_val = []
    RMSE = []
    score_val = []
    
    # 0.20	[64]	0.3	tanh	32	25
    
    # parameter's sample
    # weights_file = "weights_file_lstm_optimalmodel_clv.h5"
    alpha = 0.2
    sequence_length = 40
    epochs = 20
    nodes_per_layer = [32]
    dropout = 0.1
    activation = 'tanh'
    batch_size = 128
    remaining_sensors = remaining_sensors
    # create model
    input_shape = (sequence_length, len(remaining_sensors))    
    # Data prepration
    X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)

    # create sequences train, test
    train_array = gen_data_wrapper(X_train_interim, sequence_length,remaining_sensors)
    label_array = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'])

    test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
               for unit_nr in X_test_interim['Unit'].unique())
    
    test_array = np.concatenate(list(test_gen)).astype(np.float32)
    test_rul = rul_piecewise_fct(y_test,rul_piecewise)
    print(train_array.shape, label_array.shape, test_array.shape)
    model = model_lstm_1layer(input_shape, nodes_per_layer[0], dropout, activation)
       
    # Model fitting
    cb = EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)
    with tf.device('/device:GPU:0'):
        start_time = time.time()
        weights_file = model.get_weights()
        model.set_weights(weights_file)  # reset optimizer and node weights before every training iteration
        history = model.fit(train_array, label_array,
                                validation_data=(test_array, test_rul),
                                epochs=epochs,
                                batch_size=batch_size,
                                callbacks=[cb],
                                verbose=1)
        end_time = time.time()
        
        training_time = end_time - start_time
        mse.append(history.history['val_loss'][-1])

        y_hat_val_split = model.predict(test_array)
        R2_val.append(r2_score(test_rul, y_hat_val_split))
        RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
        score_val.append(compute_s_score(test_rul, y_hat_val_split))
            
        
    #  append results
    d = {'RMSE' :np.mean(RMSE), 'std_RMSE' :np.std(RMSE),
         'S_score' :np.mean(score_val), 'std_S_score' :np.std(score_val),
         'MSE':np.mean(mse), 'std_MSE':np.std(mse),
         'nodes':str(nodes_per_layer), 'dropout':dropout, 
         'activation':activation, 'batch_size':batch_size, 'TW' : sequence_length, 
         'time': training_time}

#     results = results.append(pd.DataFrame(d, index=[0]), ignore_index=True)
    results002 = pd.concat([results002, pd.DataFrame(d, index=[0])], ignore_index=True)
    results002.to_csv('results/base/fd002.csv')

(43619, 40, 3) (43619, 1) (259, 40, 3)
Epoch 1/20
341/341 [==============================] - 4s 9ms/step - loss: 2388.4629 - val_loss: 1790.1936
Epoch 2/20
341/341 [==============================] - 3s 8ms/step - loss: 855.0757 - val_loss: 426.8504
Epoch 3/20
341/341 [==============================] - 4s 11ms/step - loss: 476.2307 - val_loss: 307.6125
Epoch 4/20
341/341 [==============================] - 3s 8ms/step - loss: 416.4417 - val_loss: 340.2688
Epoch 5/20
341/341 [==============================] - 3s 10ms/step - loss: 383.9802 - val_loss: 280.0737
Epoch 6/20
341/341 [==============================] - 3s 9ms/step - loss: 371.8405 - val_loss: 288.0869
Epoch 7/20
341/341 [==============================] - 3s 10ms/step - loss: 360.8690 - val_loss: 284.5528
Epoch 8/20
341/341 [==============================] - 4s 11ms/step - loss: 356.3782 - val_loss: 276.4583
Epoch 9/20
341/341 [==============================] - 3s 8ms/step - loss: 343.8793 - val_loss: 275.4338
Epoch 10/20
341/341

In [8]:
results002

,RMSE,std_RMSE,S_score,std_S_score,MSE,std_MSE,nodes,dropout,activation,batch_size,TW,time
0,16.544660,0.0,1329.995532,0.0,344.317139,0.0,[32],0.1,tanh,128,40,46.287470
1,16.113632,0.0,1322.375885,0.0,259.649139,0.0,[32],0.1,tanh,128,40,59.768505
2,17.371607,0.0,1845.161542,0.0,346.541779,0.0,[32],0.1,tanh,128,40,70.753855
3,20.531673,0.0,2653.026571,0.0,458.796783,0.0,[32],0.1,tanh,128,40,21.934217
4,19.414798,0.0,3142.219070,0.0,382.160461,0.0,[32],0.1,tanh,128,40,33.085212


### FD003 <a class="anchor" id="fd003"></a>

In [9]:
# Load data
# Load data and preprocess
train, test, y_test = prepare_data('FD003.txt')
print(train.shape, test.shape, y_test.shape)
sensor_names = ['T20','T24','T30','T50','P20','P15','P30','Nf','Nc','epr','Ps30','phi',
                    'NRf','NRc','BPR','farB','htBleed','Nf_dmd','PCNfR_dmd','W31','W32']
# remaining_sensors = ["Ps30", "NRc", "W31", "BPR"] # selection based on main_notebook [1] "Nf" [1] "Ps30" [1] "phi"
remaining_sensors = ["Ps30", "NRc", "W31"] # selection based on main_notebook [1] "Nf" [1] "Ps30" [1] "phi"
drop_sensors = [element for element in sensor_names if element not in remaining_sensors]

rul_piecewise = 125
train['RUL'].clip(upper=rul_piecewise, inplace=True)

(24720, 27) (16596, 26) (100, 1)


In [10]:
%%time

# 	RMSE	std_RMSE	S_score	std_S_score	MSE	std_MSE	alpha	nodes	dropout	activation	batch_size	TW
# 43	15.404917	0.0	8.255290e+02	0.0	237.311462	0.0	0.10	[64]	0.2	tanh	32	35
results003 = pd.DataFrame()

for SEED in range(5):  
    tf.random.set_seed(SEED)
    mse = []
    R2_val = []
    RMSE = []
    score_val = []
    
    # 0.20	[64]	0.3	tanh	32	25
    
    # parameter's sample
    # weights_file = "weights_file_lstm_optimalmodel_clv.h5"
    alpha = 0.1
    sequence_length = 35
    epochs = 20
    nodes_per_layer = [64]
    dropout = 0.2
    activation = 'tanh'
    batch_size = 32
    remaining_sensors = remaining_sensors
    # create model
    input_shape = (sequence_length, len(remaining_sensors))    
    # Data prepration
    X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)

    # create sequences train, test
    train_array = gen_data_wrapper(X_train_interim, sequence_length,remaining_sensors)
    label_array = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'])

    test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
               for unit_nr in X_test_interim['Unit'].unique())
    
    test_array = np.concatenate(list(test_gen)).astype(np.float32)
    test_rul = rul_piecewise_fct(y_test,rul_piecewise)
    print(train_array.shape, label_array.shape, test_array.shape)
    model = model_lstm_1layer(input_shape, nodes_per_layer[0], dropout, activation)
       
    # Model fitting
    cb = EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)
    with tf.device('/device:GPU:0'):
        start_time = time.time()
        weights_file = model.get_weights()
        model.set_weights(weights_file)  # reset optimizer and node weights before every training iteration
        history = model.fit(train_array, label_array,
                                validation_data=(test_array, test_rul),
                                epochs=epochs,
                                batch_size=batch_size,
                                callbacks=[cb],
                                verbose=1)
        end_time = time.time()
        
        training_time = end_time - start_time
        mse.append(history.history['val_loss'][-1])

        y_hat_val_split = model.predict(test_array)
        R2_val.append(r2_score(test_rul, y_hat_val_split))
        RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
        score_val.append(compute_s_score(test_rul, y_hat_val_split))
            
        
    #  append results
    d = {'RMSE' :np.mean(RMSE), 'std_RMSE' :np.std(RMSE),
         'S_score' :np.mean(score_val), 'std_S_score' :np.std(score_val),
         'MSE':np.mean(mse), 'std_MSE':np.std(mse),
         'nodes':str(nodes_per_layer), 'dropout':dropout, 
         'activation':activation, 'batch_size':batch_size, 'TW' : sequence_length, 
         'time': training_time}

#     results = results.append(pd.DataFrame(d, index=[0]), ignore_index=True)
    results003 = pd.concat([results003, pd.DataFrame(d, index=[0])], ignore_index=True)
    results003.to_csv('results/base/fd003.csv')

(21320, 35, 3) (21320, 1) (100, 35, 3)
Epoch 1/20
667/667 [==============================] - 7s 9ms/step - loss: 1924.9816 - val_loss: 1447.5288
Epoch 2/20
667/667 [==============================] - 5s 8ms/step - loss: 504.8188 - val_loss: 690.5407
Epoch 3/20
667/667 [==============================] - 5s 8ms/step - loss: 418.0533 - val_loss: 551.8301
Epoch 4/20
667/667 [==============================] - 5s 7ms/step - loss: 377.1380 - val_loss: 493.5085
Epoch 5/20
667/667 [==============================] - 6s 8ms/step - loss: 345.7000 - val_loss: 556.4213
Epoch 6/20
667/667 [==============================] - 5s 7ms/step - loss: 338.7039 - val_loss: 357.1455
Epoch 7/20
667/667 [==============================] - 5s 8ms/step - loss: 327.7131 - val_loss: 350.6089
Epoch 8/20
667/667 [==============================] - 5s 8ms/step - loss: 312.1043 - val_loss: 348.8107
Epoch 9/20
667/667 [==============================] - 5s 7ms/step - loss: 305.1341 - val_loss: 322.0183
Epoch 10/20
667/667 [==

In [11]:
results003

,RMSE,std_RMSE,S_score,std_S_score,MSE,std_MSE,nodes,dropout,activation,batch_size,TW,time
0,15.747161,0.0,716.191246,0.0,247.973083,0.0,[64],0.2,tanh,32,35,108.887381
1,16.011698,0.0,945.740418,0.0,317.958740,0.0,[64],0.2,tanh,32,35,97.042535
2,15.916034,0.0,1086.238069,0.0,253.320114,0.0,[64],0.2,tanh,32,35,119.233871
3,16.051283,0.0,1241.762548,0.0,257.643677,0.0,[64],0.2,tanh,32,35,106.356065
4,15.495918,0.0,1001.098870,0.0,279.511597,0.0,[64],0.2,tanh,32,35,110.428248


### FD001 <a class="anchor" id="fd001"></a>

In [12]:
# Load data
# Load data and preprocess
train, test, y_test = prepare_data('FD001.txt')
print(train.shape, test.shape, y_test.shape)
sensor_names = ['T20','T24','T30','T50','P20','P15','P30','Nf','Nc','epr','Ps30','phi',
                    'NRf','NRc','BPR','farB','htBleed','Nf_dmd','PCNfR_dmd','W31','W32']
# remaining_sensors = ["Ps30", "Nf", "P30", "BPR"] # selection based on main_notebook 
remaining_sensors = ["Ps30", "Nf", "P30"] # selection based on main_notebook 
drop_sensors = [element for element in sensor_names if element not in remaining_sensors]

rul_piecewise = 125
train['RUL'].clip(upper=rul_piecewise, inplace=True)

(20631, 27) (13096, 26) (100, 1)


In [13]:
%%time

# RMSE	std_RMSE	S_score	std_S_score	MSE	std_MSE	alpha	nodes	dropout	activation	batch_size	TW
# 21	13.768992	0.0	3.830708e+02	0.0	189.585159	0.0	0.10	[128]	0.2	tanh	64	30
results001 = pd.DataFrame()

for SEED in range(5):  
    set_seed(SEED)
    mse = []
    R2_val = []
    RMSE = []
    score_val = []
    
    # 0.20	[64]	0.3	tanh	32	25
    
    # parameter's sample
    # weights_file = "weights_file_lstm_optimalmodel_clv.h5"
    alpha = 0.1
    sequence_length = 30
    epochs = 20
    nodes_per_layer = [128]
    dropout = 0.2
    activation = 'tanh'
    batch_size = 64
    remaining_sensors = remaining_sensors
    # create model
    input_shape = (sequence_length, len(remaining_sensors))    
    # Data prepration
    X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)

    # create sequences train, test
    train_array = gen_data_wrapper(X_train_interim, sequence_length,remaining_sensors)
    label_array = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'])

    test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
               for unit_nr in X_test_interim['Unit'].unique())
    
    test_array = np.concatenate(list(test_gen)).astype(np.float32)
    test_rul = rul_piecewise_fct(y_test,rul_piecewise)
    print(train_array.shape, label_array.shape, test_array.shape)
    model = model_lstm_1layer(input_shape, nodes_per_layer[0], dropout, activation)
       
    # Model fitting
    cb = EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)
    with tf.device('/device:GPU:0'):
        start_time = time.time()
        weights_file = model.get_weights()
        model.set_weights(weights_file)  # reset optimizer and node weights before every training iteration
        history = model.fit(train_array, label_array,
                                validation_data=(test_array, test_rul),
                                epochs=epochs,
                                batch_size=batch_size,
                                callbacks=[cb],
                                verbose=1)
        end_time = time.time()
        
        training_time = end_time - start_time
        mse.append(history.history['val_loss'][-1])

        y_hat_val_split = model.predict(test_array)
        R2_val.append(r2_score(test_rul, y_hat_val_split))
        RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
        score_val.append(compute_s_score(test_rul, y_hat_val_split))
            
        
    #  append results
    d = {'RMSE' :np.mean(RMSE), 'std_RMSE' :np.std(RMSE),
         'S_score' :np.mean(score_val), 'std_S_score' :np.std(score_val),
         'MSE':np.mean(mse), 'std_MSE':np.std(mse),
         'nodes':str(nodes_per_layer), 'dropout':dropout, 
         'activation':activation, 'batch_size':batch_size, 'TW' : sequence_length, 
         'time': training_time}

#     results = results.append(pd.DataFrame(d, index=[0]), ignore_index=True)
    results001 = pd.concat([results001, pd.DataFrame(d, index=[0])], ignore_index=True)
    results001.to_csv('results/base/fd001.csv')

(17731, 30, 3) (17731, 1) (100, 30, 3)
Epoch 1/20
278/278 [==============================] - 8s 22ms/step - loss: 2104.4343 - val_loss: 1636.0094
Epoch 2/20
278/278 [==============================] - 5s 18ms/step - loss: 1424.5371 - val_loss: 814.5938
Epoch 3/20
278/278 [==============================] - 5s 17ms/step - loss: 803.5678 - val_loss: 682.2605
Epoch 4/20
278/278 [==============================] - 4s 16ms/step - loss: 556.0916 - val_loss: 350.6610
Epoch 5/20
278/278 [==============================] - 5s 18ms/step - loss: 405.3134 - val_loss: 556.1797
Epoch 6/20
278/278 [==============================] - 5s 19ms/step - loss: 374.6635 - val_loss: 329.7100
Epoch 7/20
278/278 [==============================] - 5s 17ms/step - loss: 344.5364 - val_loss: 253.1703
Epoch 8/20
278/278 [==============================] - 5s 17ms/step - loss: 321.8900 - val_loss: 274.8988
Epoch 9/20
278/278 [==============================] - 5s 17ms/step - loss: 304.8224 - val_loss: 298.3569
Epoch 10/20
2

In [14]:
results001

,RMSE,std_RMSE,S_score,std_S_score,MSE,std_MSE,nodes,dropout,activation,batch_size,TW,time
0,14.865401,0.0,393.923223,0.0,570.080994,0.0,[128],0.2,tanh,64,30,89.534018
1,15.955759,0.0,413.238111,0.0,254.586243,0.0,[128],0.2,tanh,64,30,100.249789
2,15.020641,0.0,367.751512,0.0,255.703674,0.0,[128],0.2,tanh,64,30,74.453083
3,14.897409,0.0,356.586710,0.0,249.360062,0.0,[128],0.2,tanh,64,30,73.219218
4,15.300330,0.0,432.614388,0.0,234.100113,0.0,[128],0.2,tanh,64,30,83.403499
